In [ ]:
import decimal, numpy
d = decimal.Decimal('1.1')
a = numpy.array([d,d,d],dtype=numpy.dtype(decimal.Decimal))
print(a.dtype)
print(type(a[1]))

In [ ]:
s = [['123.123','23'],['2323.212','123123.21312']]
decimal_s = [[decimal.Decimal(x) for x in y] for y in s]
ss = numpy.array(decimal_s)
print(type(ss))
print(ss.dtype)
print(type(ss[0,0]))

# 오버로딩 처리


In [1]:
import overload
dir(overload)

['TestOverload',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 'functools',
 'overload',
 'types',
 'unittest']

In [2]:
from overload import overload
class A(object):
    @overload
    def method(self, a):
        return 'a'
    @method.add
    def method(self, a, b):
        return a+b

a = A()
print(a.method(1))

print(a.method(1, 2))

a
3


## 디스크립터 처리

In [26]:
## 직접 인스턴스 내의 __dict__로 접근해서 처리

class Descriptor :
    def __init__(self, name) :
        self.name = name
        
    def __get__(self,instance, owner) :
        print("__get__")
        return instance.__dict__[self.name]
    
    def __set__(self,instance, value) :
        print("__set__")
        instance.__dict__[self.name] = value
        
class A :
    name = Descriptor("name")
    def __init__(self, name) :
        self.name = name
   
        
a = A("moon")

print(A.__dict__)
print(a.__dict__)
a.name = "dahl"
print(a.name)

__set__
{'__doc__': None, '__module__': '__main__', '__init__': <function A.__init__ at 0x10481ef28>, 'name': <__main__.Descriptor object at 0x1046a46d8>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>}
{'name': 'moon'}
__set__
__get__
dahl


In [30]:
## getattr, setattr 로 처리시 instance.name으로 접근하면 무한순환에러가 발생함
## instance 이름을 "_" 넣고 처리

class Descriptor :
    def __init__(self, name) :
        self.name = "_"+name
        
    def __get__(self,instance, owner) :
        print("__get__")
        return getattr(instance, self.name )
    
    def __set__(self,instance, value) :
        print("__set__")
        setattr(instance, self.name, value)
        
class A :
    name = Descriptor("name")
    def __init__(self, name) :
        print("__init__")
        #기존 name에 먼저 A  클래스에 존재 함 
        self.name = name
   
        
a = A("moon")

print(A.__dict__)
print(a.__dict__)
a.name = "dahl"
print(a.name)

__init__
__set__
{'__doc__': None, '__module__': '__main__', '__init__': <function A.__init__ at 0x104833730>, 'name': <__main__.Descriptor object at 0x104851be0>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>}
{'_name': 'moon'}
__set__
__get__
dahl


In [3]:
help(setattr)

Help on built-in function setattr in module builtins:

setattr(obj, name, value, /)
    Sets the named attribute on the given object to the specified value.
    
    setattr(x, 'y', v) is equivalent to ``x.y = v''



In [10]:
import pprint
pprint.pprint(A.__dict__.items())

for i,v in A.__dict__.items() :
    if i == "__dict__" :
        print(dir(v))
        print(type(v))

dict_items([('__doc__', None), ('__module__', '__main__'), ('name', <__main__.Descriptor object at 0x10466d668>), ('__dict__', <attribute '__dict__' of 'A' objects>), ('__weakref__', <attribute '__weakref__' of 'A' objects>)])
['__class__', '__delattr__', '__delete__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__objclass__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__set__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
<class 'getset_descriptor'>


In [20]:
class B :
    def __init__(self, name) :
        self.name = name
        
    def __getattr__(self,name) :
        print(" __getattr__")
        
        setattr(self,name,"default")
        return self.__dict__[name]
    
b = B("dahl")
print(b.name)

print(getattr(b,"__init__"))

# 없는 속성을 찾을 경우
# __getattr__를 읽어 처리함
print(b.age)

dahl
<bound method B.__init__ of <__main__.B object at 0x104835d30>>
 __getattr__
default
